In [2]:
text="This paper further discusses the RAG training, including RAGwith/without datastore update. Then, we introduce the application of RAG in representative natural language processing tasks and industrial scenarios. Finally, this paper discusses the future directions and challenges of RAG for promoting its development"
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))
print ('---')
print(text)
print("lengthText:", len(text))
print('---')
print(tokens)
print("lengthTokens:", len(tokens))

---
This paper further discusses the RAG training, including RAGwith/without datastore update. Then, we introduce the application of RAG in representative natural language processing tasks and industrial scenarios. Finally, this paper discusses the future directions and challenges of RAG for promoting its development
lengthText: 314
---
[84, 104, 105, 115, 32, 112, 97, 112, 101, 114, 32, 102, 117, 114, 116, 104, 101, 114, 32, 100, 105, 115, 99, 117, 115, 115, 101, 115, 32, 116, 104, 101, 32, 82, 65, 71, 32, 116, 114, 97, 105, 110, 105, 110, 103, 44, 32, 105, 110, 99, 108, 117, 100, 105, 110, 103, 32, 82, 65, 71, 119, 105, 116, 104, 47, 119, 105, 116, 104, 111, 117, 116, 32, 100, 97, 116, 97, 115, 116, 111, 114, 101, 32, 117, 112, 100, 97, 116, 101, 46, 32, 84, 104, 101, 110, 44, 32, 119, 101, 32, 105, 110, 116, 114, 111, 100, 117, 99, 101, 32, 116, 104, 101, 32, 97, 112, 112, 108, 105, 99, 97, 116, 105, 111, 110, 32, 111, 102, 32, 82, 65, 71, 32, 105, 110, 32, 114, 101, 112, 114, 101, 

In [3]:
def get_stats(ids):
    counts ={}
    for pair in zip(ids, ids[1:]):
        counts[pair]= counts.get(pair,0)+1
        return counts
stats=get_stats(tokens)
print(stats)
#print(sorted((V,K)for K,V in stats.items()), reverse=True())        

{(84, 104): 1}


In [4]:
top_pair= max(stats, key=stats.get)
top_pair

(84, 104)

In [5]:
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i< len(ids):
        if i <len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i+=2
        else:
            newids.append(ids[i])
            i+=1
    return newids

In [6]:
print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))

[5, 6, 99, 9, 1]


In [7]:
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print('length:', len(tokens2))

[256, 105, 115, 32, 112, 97, 112, 101, 114, 32, 102, 117, 114, 116, 104, 101, 114, 32, 100, 105, 115, 99, 117, 115, 115, 101, 115, 32, 116, 104, 101, 32, 82, 65, 71, 32, 116, 114, 97, 105, 110, 105, 110, 103, 44, 32, 105, 110, 99, 108, 117, 100, 105, 110, 103, 32, 82, 65, 71, 119, 105, 116, 104, 47, 119, 105, 116, 104, 111, 117, 116, 32, 100, 97, 116, 97, 115, 116, 111, 114, 101, 32, 117, 112, 100, 97, 116, 101, 46, 32, 256, 101, 110, 44, 32, 119, 101, 32, 105, 110, 116, 114, 111, 100, 117, 99, 101, 32, 116, 104, 101, 32, 97, 112, 112, 108, 105, 99, 97, 116, 105, 111, 110, 32, 111, 102, 32, 82, 65, 71, 32, 105, 110, 32, 114, 101, 112, 114, 101, 115, 101, 110, 116, 97, 116, 105, 118, 101, 32, 110, 97, 116, 117, 114, 97, 108, 32, 108, 97, 110, 103, 117, 97, 103, 101, 32, 112, 114, 111, 99, 101, 115, 115, 105, 110, 103, 32, 116, 97, 115, 107, 115, 32, 97, 110, 100, 32, 105, 110, 100, 117, 115, 116, 114, 105, 97, 108, 32, 115, 99, 101, 110, 97, 114, 105, 111, 115, 46, 32, 70, 105, 110, 97,

In [8]:
vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

In [9]:
merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
print(f"merging {pair} into a new token {idx}")
ids = merge(ids, pair, idx)
merges [pair] = idx    

merging (84, 104) into a new token 275


In [10]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):2f}X")

tokens length: 314
ids length: 312
compression ratio: 1.006410X


ecoding

In [11]:
vocab = {idx: bytes ([idx] )for idx in range(256) }
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab [p0] + vocab [p1]

In [12]:
def decode (ids): 
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text
print(decode([33]))

!


ncoding

In [13]:
def encode(text):
    tokens =  list(text.encode("utf-8"))
    while True:
        stats =  get_stats(tokens)
        pair = min(stats, key= lambda p: merges.get(p, float ("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens =  merge(tokens, pair, idx)
    return tokens
print(encode("Have a nice day!")) 

[72, 97, 118, 101, 32, 97, 32, 110, 105, 99, 101, 32, 100, 97, 121, 33]


ara verificar se o decode e encode estÃ¡ correto

In [14]:
text1 = decode(encode(text))
print(text1 == text)
#verificando outro texto que o tokenizer nÃ£o conhece
valtext = "This acceleration was largely driven by three synergistic trends. First, large amounts of spoken and written material became widely available through the auspices of the Linguistic Data Consortium (LDC)."
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True
True


ivide o texto em uma lista de texto e cada um Ã© processado diferente

In [15]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
print(re.findall(gpt2pat, "How is going. I'm great"))  

['How', ' is', ' going', '.', ' I', "'m", ' great']


rabalha com os espaÃ§os 

In [16]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("       hello world!!!!"))

[220, 220, 220, 220, 220, 220, 23748, 995, 13896]


In [17]:
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("       hello world!!!!"))

[996, 24748, 1917, 17523]


In [18]:
import os, json

In [19]:
with open('encoder.json', 'r') as f:
    encoder = json.load(f)
    
with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
print(len(encoder))

50257
